<a href="https://colab.research.google.com/github/otanet/Quantum_Computing_Annealing_Machine_20211209/blob/main/coloring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Coloring

Amplify を用いたグラフ彩色問題の解法について解説します。


## グラフ彩色問題の定式化


グラフ彩色問題とは、あるグラフが与えられたときに、与えられた制約条件の下でその頂点などに色を割り当てる問題です。最も典型的な問題は頂点に対して隣接する頂点同士別の色で塗り分ける問題です。

![200px-Petersen_graph_3-coloring.svg.png](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Petersen_graph_3-coloring.svg/200px-Petersen_graph_3-coloring.svg.png)

平面グラフ (地図) においては、隣接する領域同士においてどんな場合でも四色あれば塗り分けられるという [四色定理](https://en.wikipedia.org/wiki/Four_color_theorem) が知られています。しかしその塗り分け方法については自明ではありません。

グラフ彩色問題にはいくつかの応用例が知られており、例えば、会議室・機械・タスクなどの割り当てに関するスケジューリング問題や、コンパイラによるレジスタ割り付け、携帯電話網における周波数割り当て等が挙げられます。今回は日本の都道府県に対して、イジングマシンを用いた塗り分けを行います。

イジングマシンを用いるためにグラフの彩色状態に対して二値で表現する方法を考えます。次のように、各領域に対して四変数を用いて $0$ または $1$ を割り当てることで表現が可能です。

| Region | Red | Green | Blue | Yellow |
|--------|-----|-------|------|--------|
| 1      | 0   | 0     | 1    | 0      |
| 2      | 0   | 1     | 0    | 0      |
| 3      | 0   | 0     | 1    | 0      |
| 4      | 1   | 0     | 0    | 0      |

    
この例では領域 $1$ に青、領域 $2$ に緑、領域 $3$ に青、領域 $4$ に赤を割り当てることを意味しています。上記の表における各変数を領域のインデックス $i$ と色のインデックス $c$ を用いて、$q_{i,c}$ と表すことにします。必要な変数の数は領域数 $N$、 色数 $C$ に対して $NC$ となります。

塗り分け問題の定義から変数の間には次の制約条件が課せられます。

* 一領域を一色で塗る
* 隣接する領域に対して同色で塗らない

これらについて定式化を行うと次のように表されます。

**制約条件**

$$
    \sum_{c = 0}^{C-1}{ q_{i,c} } = 1 \quad \text{for} \; i \in \left\{0, 1, \ldots, N - 1 \right\} \\
    q_{i,c} q_{j,c} = 0 \quad \text{for} \; (i, j) \in E, \; c \in \{0, 1, \ldots, C-1\}
$$

ここで $E$ はグラフ上の隣接している領域の組の集合を表します。後でプログラムコード化する都合により、変数のインデックスは $0$ からスタートしていることに注意してください。


## 問題の作成

日本地図を扱うために Python の [japanmap](https://pypi.org/project/japanmap/) モジュールを使用します。都道府県コード ( $1 \sim{} 47$) を用いて都道府県名や隣接情報などを取得できます。

まずは色の定義を行い、変数テーブルを用意します。

In [ ]:
! pip install -q amplify japanmap

In [ ]:
from amplify import BinarySymbolGenerator
import japanmap as jm

colors = ["red", "green", "blue", "yellow"]
num_colors = len(colors)
num_region = len(jm.pref_names) - 1  # 都道府県数を取得

gen = BinarySymbolGenerator()
q = gen.array(num_region, num_colors)

In [ ]:
q

次に制約条件を作成します。$1$ 領域を $1$ 色で塗る One-hot 制約 $\sum_{c = 0}^{C-1}{ q_{i,c} } = 1$ は `amplify.constraint.one_hot` 関数を用いて作成することができます。
隣りあう領域を異なる色で塗る制約 $q_{i,c} q_{j,c} = 0$ については、 `amplify.constraint.equal_to` 関数を用います。
`japanmap.adjacent` 関数に都道府県コードを入力するとその都道府県に隣接する都道府県コードのリストを取得できます。
`q` に対するインデックスと都道府県コードには $1$ だけ差分があるため注意してください。

In [ ]:
from amplify import sum_poly
from amplify.constraint import one_hot, equal_to

# 各領域が1色で塗られる制約
reg_constraints = [one_hot(q[i]) for i in range(num_region)]

# 隣接する領域を異なる色で塗る制約
adj_constraints = [
    # 都道府県コードと配列インデックスは1ずれているので注意
    equal_to(q[i, c] * q[j - 1, c], 0)
    for i in range(num_region)
    for j in jm.adjacent(i + 1)  # j: 隣接している都道府県コード
    if i + 1 < j
    for c in range(num_colors)
]

constraints = sum(reg_constraints) + sum(adj_constraints)

## イジングマシンの実行

イジングマシンのクライアントを作成しパラメータを設定します。その後ソルバーを作成しクライアントを設定します。

In [ ]:
from amplify import Solver
from amplify.client import FixstarsClient

client = FixstarsClient()
client.parameters.timeout = 5000  # タイムアウト5秒
client.token = "トークンを入力してください"

solver = Solver(client)

制約条件から論理模型を作成し、次のようにしてイジングマシンを実行し結果を取得します。

In [ ]:
from amplify import BinaryQuadraticModel

model = BinaryQuadraticModel(constraints)
result = solver.solve(model)
if len(result.solutions) == 0:
    raise RuntimeError("Any one of constraints is not satisfied.")

values = result.solutions[0].values

### Note

もし ``result.solutions`` オブジェクトが空のリストの場合、制約条件を満たす解が得られなかったことを意味します。この場合はイジングマシンのパラメータの変更が必要です。

## 結果の解析

`values` は入力変数と解の値のマッピングを表す辞書です。そのままでは評価しづらいので、次のようにして変数テーブル `q` と同一の形式にデコードします。

In [ ]:
q_values = q.decode(values)

結果を ``{都道府県名: 色}`` の形式に変換します。まずは ``q_values`` の各行のうち値が1のインデックスを `numpy` の `np.where` 関数を用いて取得します。
その後、`japanmap.pref_names` を用いて都道府県コードを都道府県名に変換し、都道府県名と色を格納した辞書を作成します。


In [ ]:
import numpy as np

color_indices = np.where(q_values == 1)[1]
color_map = {
    jm.pref_names[i + 1]: colors[color_indices[i]] for i in range(len(color_indices))
}

In [ ]:
color_map

最後に得られた塗り分けを表示します。次のようにしてプロットされます。

In [ ]:
! pip install -q matplotlib
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = 6, 6
plt.imshow(jm.picture(color_map))
plt.show()

## プログラミング課題

### 問題1
色数を3色にすることを試みてみましょう。
日本地図は3色で塗り分けられないので実行可能解は出力されず、
デフォルトの設定では結果配列の長さが 0 になりますが、
`solver.filter_solution=False` とすることで制約条件を満たしていない解も出力することができます。
どのような解が出力されるか確認してください。

### 問題2
上下左右に隣り合うどの $2$ つのマスについてもそのマスに入っている $2$ 数が互いに素になるように、
$1$ から $20$ までの数字を $4\times 5$ のマス目に $1$ つずつ入れる方法を求めるプログラムを書いてください。